In [1]:
import os

import rangekeeper as rk

In [2]:
# Authenticate with Speckle
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN')) # Note if you run this notebook yourself, you'll need to set this environment variable



 SpeckleClient( server: https://speckle.xyz, authenticated: True )


In [3]:
stream_id = "3f40d86240"
commit_id = speckle.get_latest_commit_id(stream_id)

In [4]:
# Load the design scenario
model = speckle.get_commit(stream_id=stream_id) # Providing no commit_id defaults to the latest commit
model

Base(id: 25b3551c0b32e8044de35043525ae5d3, speckle_type: Base, totalChildrenCount: 2074)

In [5]:
from IPython.display import IFrame
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)

In [6]:
# Let's identify the root members of the scenario:
roots = model.get_dynamic_member_names()
roots

['@scenario', '@context']

In [7]:
IFrame("https://speckle.xyz/streams/{0}/objects/{1}".format(stream_id, model['@scenario']['id']), width='100%', height=800)

In [8]:
# Return any Speckle Objects that have 'entityId's:
speckle_entities = rk.api.Speckle.parse(model['@scenario'])

# Convert the first object (root) into a Rangekeeper Entity:
root_assembly = rk.api.Speckle.to_entity(speckle_entities[0])

# (We can check that it is an Assembly:)
isinstance(root_assembly, rk.graph.Assembly)

True

In [9]:
# Get the "BuildingA" Assembly:
buildingA = [node for node in root_assembly.nodes if node.name == 'BuildingA'][0]
print('BuildingA: {0}'.format(buildingA))

BuildingA: Assembly: BuildingA
Type: building
Entities: [(Assembly: BuildingA (Type: building), {}), (Entity: BuildingA.Residential (Type: space), {}), (Entity: BuildingA.Retail (Type: space), {}), (Entity: BuildingA.Parking (Type: space), {}), (Entity: BuildingA.Office (Type: space), {}), (Entity: BuildingA.Core (Type: space), {})]
Relationships: [(Assembly: BuildingA (Type: building), Entity: BuildingA.Residential (Type: space), 'spatiallyContains'), (Assembly: BuildingA (Type: building), Entity: BuildingA.Retail (Type: space), 'spatiallyContains'), (Assembly: BuildingA (Type: building), Entity: BuildingA.Parking (Type: space), 'spatiallyContains'), (Assembly: BuildingA (Type: building), Entity: BuildingA.Office (Type: space), 'spatiallyContains'), (Assembly: BuildingA (Type: building), Entity: BuildingA.Core (Type: space), 'spatiallyContains'), (Entity: BuildingA.Core (Type: space), Entity: BuildingA.Residential (Type: space), 'services'), (Entity: BuildingA.Core (Type: space), Enti

In [10]:
# Get all relatives of BuildingA where BuildingA is the source of
# a 'spatiallyContains' relationship:
buildingA_containment = buildingA.get_relatives(outgoing=True, relationship_type='spatiallyContains')
print('BuildingA Containment: \n {0}\n'.format(buildingA_containment))

BuildingA Containment: 
 [Entity: BuildingA.Residential (Type: space), Entity: BuildingA.Retail (Type: space), Entity: BuildingA.Parking (Type: space), Entity: BuildingA.Office (Type: space), Entity: BuildingA.Core (Type: space)]



In [11]:
develop = root_assembly.develop(
    name='design_scenario',
    type='design_scenario')

In [12]:
develop.plot(
    notebook=True,
    hierarchical_layout=False,
    display=False)

design_scenario.html
